In [1]:
import pdfplumber
import pandas as pd
import re

# Ouvrir le PDF
pdf_file = "nicewicz_chemcommun_2016_crop.pdf"

In [2]:
# Variables pour stocker les données
data = []
columns = [
    "Abbreviation", "L(abs,max), nm", "tf (ns)", "(phi)f", "(phi)ISC", "E(0,0; S1)", "E(0,0; T1)", 
    "Ered", "Eox", "Ered, S1", "Eox, S1", "Ered, T1", "Eox, T1"
]


In [13]:
with pdfplumber.open(pdf_file, laparams={"all_texts": True}) as pdf:
    for page in pdf.pages:
        # Extraction brute
        text = page.extract_text(x_tolerance=1, y_tolerance=1)
       
        # Détection des lignes du tableau
        rows = re.findall(
            r"([A-Za-z0-9+~\-]+)\s+([0-9.~]+)\s+([0-9.~]+)\s+([0-9.~]+)\s+([0-9.~]+)\s+([0-9.~]+)\s+([0-9.~]+)\s+([-0-9.~]+)\s+([-0-9.~]+)\s+([-0-9.~]+)\s+([-0-9.~]+)\s+([-0-9.~]+)\s+([-0-9.~]+)",
            text.replace("\n", " ")
        )
        
        # Ajouter les lignes au tableau de données
        for row in rows:
            data.append(row)

CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox


In [ ]:
# Vérifier que des données ont été extraites
if data:
    # Créer le DataFrame
    df = pd.DataFrame(data, columns=columns)

    # Exporter en CSV
    output_file = "/mnt/data/photophysical_properties_corrected.csv"
    df.to_csv(output_file, index=False)
    print(f"Les données ont été exportées avec succès dans le fichier : {output_file}")
else:
    print("Aucune donnée trouvée. Vérifie le format du tableau.")

In [4]:
# Créer un DataFrame
df = pd.DataFrame(data, columns=columns)

In [6]:
# Exporter en CSV
output_file = "nicewicz_photocatalysts.csv"
df.to_csv(output_file, index=False)
print(f"Les données ont été exportées avec succès dans le fichier : {output_file}")

Les données ont été exportées avec succès dans le fichier : nicewicz_photocatalysts.csv


In [17]:
import camelot
import pandas as pd

# Chemin vers le PDF
pdf_file = "nicewicz_chemcommun_2016_crop.pdf"

# Extraction des tableaux avec camelot
tables = camelot.read_pdf(pdf_file, pages='all', flavor='stream')  # ou 'lattice' si tables bien délimitées

# Afficher le nombre de tables détectées
print(f"{len(tables)} tableaux extraits.")

# Concaténer tous les tableaux détectés en un seul DataFrame
df_all = pd.concat([table.df for table in tables], ignore_index=True)

# Exporter en CSV
output_file = "photophysical_properties_camelot.csv"
df_all.to_csv(output_file, index=False)

print(f"Données exportées dans : {output_file}")


CropBox missing from /Page, defaulting to MediaBox


0 tableaux extraits.


/Users/victorcarre/Library/Python/3.9/lib/python/site-packages/camelot/parsers/base.py:124: UserWarning: page-1 is image-based, camelot only works on text-based pages.
  warnings.warn(


ValueError: No objects to concatenate

In [23]:
from pdf2image import convert_from_path
import pytesseract
import re
import pandas as pd

pdf_file = "nicewicz_chemcommun_2016_crop.pdf"
pages = convert_from_path(pdf_file)

data = []
columns = [
    "Abbreviation", "L(abs,max), nm", "tf (ns)", "(phi)f", "(phi)ISC", "E(0,0; S1)", "E(0,0; T1)", 
    "Ered", "Eox", "Ered, S1", "Eox, S1", "Ered, T1", "Eox, T1"
]

def nettoyage_texte_ligne(line):
    # Enlever caractères parasites
    line = line.replace('‘', "'").replace('’', "'").replace('—', '-')
    # Remplacer multiples espaces par un seul
    line = ' '.join(line.split())
    # Supprimer caractères non imprimables (ex: accents mal lus)
    line = ''.join(c for c in line if c.isprintable())
    return line

for i, page in enumerate(pages):
    text = pytesseract.image_to_string(page)
    # Nettoyer et rechercher les lignes tabulaires
    lines = text.split('\n')
    for line in lines:
        # Exemple de pattern simple, à ajuster selon OCR
        match = re.match(
            r"([A-Za-z0-9+~\-]+)\s+([0-9.~]+)\s+([0-9.~]+)\s+([0-9.~]+)\s+([0-9.~]+)\s+([0-9.~]+)\s+([0-9.~]+)\s+([-0-9.~]+)\s+([-0-9.~]+)\s+([-0-9.~]+)\s+([-0-9.~]+)\s+([-0-9.~]+)\s+([-0-9.~]+)", 
            line)
        if match:
            data.append(match.groups())

text = pytesseract.image_to_string(page)
print(text[:1000])  # Affiche les 1000 premiers caractères pour voir la qualité


df = pd.DataFrame(data, columns=columns)
df.to_csv("photophysical_properties_ocr.csv", index=False)
print("Extraction terminée, fichier photophysical_properties_ocr.csv créé")


SS
DCB | 290 | CT 8 tt tess tS
DCN a Co QO OY > sO Ye es ee
pea | 4a _| 49 | ove | omoes | 290 | 181 | 01 [| no +. os?

TXO ET S'S GSC OS > OS Oy OO SOO SO

reag | 40 | ‘| ‘| 10 || 246 | oo || — | —S—idY ate

ppg | =409 |) nn 0g st

aq | 36 | toe i ars 06 ir
ew

NMar [sisi ii

a

Rewer [| 37 if 0s SS—~d OSS

PheAccMer | #24 oo [| Cd oe Cr CCC CCST

x0 CS EO EC EC

a A ET

Aa m || o% |_| _286 a a

PF SE SC
a Re es QS QO OO QO CO

rR] 2D oso | 00 | om | a7 | 18 | os) | war | is | 133 | oar | 056
a Rs CO QQ

[Rh6G] 530 4.13 0.90 0.002 2.32 2.09 -1.14 +1.23 +1.18 —1.09 +0.95 —0.86


Extraction terminée, fichier photophysical_properties_ocr.csv créé
